# Compile Time Comparison
***
## Setup the Environment
***

In [1]:
import os
import sys
sys.path.insert(1, os.path.abspath(os.path.join(os.getcwd(), '../..')))

In [2]:
import time

MatPlotLib and Plotly are used for creating custom visualizations

In [3]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.offline import iplot

In [4]:
import spatialpy

***
## Create Geometries for the Domain
***

In [5]:
class Left(spatialpy.Geometry):
    def inside(self, x, on_boundary):
        if x[0] >= 0:
            return True
        return False

In [6]:
class Right(spatialpy.Geometry):
    def inside(self, x, on_boundary):
        if x[0] < 0:
            return True
        return False

***
## Create the Diffusion Reaction Model
***
This particular implementation of this model is used for debugging

In [7]:
def create_diffusion_reaction(nx, ny, model_name="diffreact_debug_test", parameter_values=None):
    # Initialize Model
    model = spatialpy.Model(model_name)
    
    # Define Domain Type IDs as constants of the Model
    model.LEFT = Left.__name__
    model.RIGHT = Right.__name__

    # System constants
    D_const = 0.01

    # Define Domain
    domain = spatialpy.Domain.create_2D_domain(
        xlim=[-1, 1], ylim=[-1, 1], nx=nx, ny=ny, type_id=model.LEFT,
        mass=1.0, nu=1.0, fixed=True,  rho0=1.0, c0=1.0, P0=1.0
    )
    
    # Add Type IDs to the Particles
    domain.set_properties(Right(), model.RIGHT)
    
    # Set Models Domain
    model.add_domain(domain)

    # Define Variables (SpatialPy.Species)
    A = spatialpy.Species(name="A", diffusion_coefficient=D_const)
    
    # Add Variables to Model
    model.add_species(A)

    # Define Parameters
    Rate1 = spatialpy.Parameter("Rate1", expression=100)
    
    # Add Parameters to Model
    # model.add_parameter(Rate1)

    # Define Reactions
    R1 = spatialpy.Reaction(name="R1", reactants={}, products={'A': 1}, rate='Rate1', restrict_to=model.LEFT)
    R2 = spatialpy.Reaction(name="R2", reactants={'A': 1}, products={}, rate='Rate1', restrict_to=model.RIGHT)
    
    # Add Reactions to Model
    # model.add_reaction([R1, R2])

    # Define Timespan
    tspan = spatialpy.TimeSpan.linspace(t=1, num_points=11, timestep_size=0.1)
    
    # Set Model Timespan
    model.timespan(tspan)
    return model

***
## Create the Compile Test
***

In [8]:
class CompileTest():

    def __init__(self, max_compile_time, verbose=0):
        self.times = []
        self.num_particles = []
        self.max_compile_time = max_compile_time
        self.verbose = verbose
        
    def build_test(self, nx, ny):
        model = create_diffusion_reaction(nx, ny)
        solver = spatialpy.Solver(model, debug_level=0)
        return model, solver
    
    def run(self):
        self.run_test()
        self.plot_results()
        
    def run_test(self):
        for i in range(2, 10000):
            num_particles = pow(i, 2)
            self.num_particles.append(num_particles)
            
            model, solver = self.build_test(nx=i, ny=i)
            
            start = time.time()
            solver.compile()
            diff = time.time()-start
            self.times.append(diff)
            
            if self.verbose >= 1:
                print(f"The model has {num_particles} particles -- Time to compile was {diff}")
                
            if diff > self.max_compile_time:
                break
                
    def plot_results(self):
        trace = go.Scatter(
            x=self.num_particles,
            y=self.times,
            mode='lines+markers',
        )
        
        layout = {"title": "Compile Time for Models with Varying Particle Counts",
                  "xaxis_title": "Number of Particles",
                  "yaxis_title": "Average Time (in seconds)"}
        
        fig = {"data":[trace], "layout":layout}

        iplot(fig)

### Instantiate the Compile Test

In [9]:
test = CompileTest(max_compile_time=60)

***
## Run the Compile Test
***

In [ ]:
%time test.run()

***
## Inspect the Solver Compilation Using `prun`
***
### Instatiate the model

In [ ]:
model = create_diffusion_reaction(40, 40)

### Instantiate the Solver

In [ ]:
solver = spatialpy.Solver(model=model)
%prun solver.compile()